In [19]:
import pandas as pd
import usi_utils
from io import StringIO
import requests

In [20]:
dtype = {
    "filepath": object,
    "dataset": object,
    "collection": object,
    "is_update": "int64",
    "update_name": object,
    "create_time": object,
    "size": "int64",
    "size_mb": "int64",
    "sample_type": object,
    "spectra_ms1": "float64",
    "spectra_ms2": "float64",
    "instrument_vendor": object,
    "instrument_model": object,
    "file_processed": object,
    "file_usi": object
}

In [21]:
def download_dataset_stats(file_format):
    response = requests.get("https://gnps-datasetcache.ucsd.edu/datasette/database/filename"
                  f".csv?_stream=on&_sort=filepath&filepath__endswith=.{file_format}&_size=max")
    response.raise_for_status()
    
    all_files = pd.read_csv(StringIO(response.text), dtype=dtype)
    all_files.to_parquet(f"../local_files/all_gnps_files/all_{file_format}_auto.parquet")
    return all_files
    

In [22]:
# the latest automatically downloaded files
ms_formats = ["mzml", "mzxml"]

files_dfs = [] 

for fformat in ms_formats:
    auto_download = []
    try:
        auto_download = download_dataset_stats(fformat)
    except:
        pass
    if len(auto_download) == 0:  # load latest downloaded version if failed
        print(f"Trying to load latest download {fformat}")
        auto_download = pd.read_parquet(f"../local_files/all_gnps_files/all_{fformat}_auto.parquet")
    if len(auto_download) == 0:  # load backup old version on fail
        print(f"Trying to load old recovery file {fformat}")
        auto_download = pd.read_csv(f"../local_files/all_gnps_files/all_{fformat}.txt", dtype=dtype)
    if len(auto_download) > 0:
        files_dfs.append(auto_download)
        
file_stats = pd.concat(files_dfs)

In [25]:
file_stats["file_usi"] = [
    usi_utils.create_simple_file_usi(path, dataset) for path, dataset in zip(file_stats["filepath"],
                                                                             file_stats["dataset"])
]
file_stats

,filepath,dataset,collection,is_update,update_name,create_time,size,size_mb,sample_type,spectra_ms1,spectra_ms2,instrument_vendor,instrument_model,file_processed,file_usi
0,MSV000073062/73062/mzml/peaklist1.mzml,MSV000073062,73062,0,NaN,2013-06-07 00:00:00,7166071,6,DEFAULT,0.0,0.0,NaN,NaN,FAILED,mzspec:MSV000073062:peaklist1
1,MSV000073062/73062/mzml/peaklist2.mzml,MSV000073062,73062,0,NaN,2013-06-07 00:00:00,8821747,8,DEFAULT,0.0,0.0,NaN,NaN,FAILED,mzspec:MSV000073062:peaklist2
2,MSV000073062/73062/mzml/peaklist3.mzml,MSV000073062,73062,0,NaN,2013-06-07 00:00:00,5751122,5,DEFAULT,0.0,0.0,NaN,NaN,FAILED,mzspec:MSV000073062:peaklist3
3,MSV000073062/73062/mzml/peaklist4.mzml,MSV000073062,73062,0,NaN,2013-06-07 00:00:00,10366660,9,DEFAULT,0.0,0.0,NaN,NaN,FAILED,mzspec:MSV000073062:peaklist4
4,MSV000073062/73062/mzml/peaklist5.mzml,MSV000073062,73062,0,NaN,2013-06-07 00:00:00,2216166,2,DEFAULT,0.0,0.0,NaN,NaN,FAILED,mzspec:MSV000073062:peaklist5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484281,MSV000091804/updates/2023-04-28_Bjosephc_21f27...,MSV000091804,peak,1,2023-04-28_Bjosephc_21f275f7,2023-04-26 03:17:00,160338738,152,DEFAULT,8275.0,97.0,Thermo Fisher Scientific,Q Exactive HF,DONE,mzspec:MSV000091804:2022081803
484282,MSV000091804/updates/2023-04-28_Bjosephc_21f27...,MSV000091804,peak,1,2023-04-28_Bjosephc_21f275f7,2023-04-26 03:16:00,152195548,145,DEFAULT,7066.0,33.0,Thermo Fisher Scientific,Q Exactive HF,DONE,mzspec:MSV000091804:2022081806
484283,MSV000091804/updates/2023-04-28_Bjosephc_262e3...,MSV000091804,peak,1,2023-04-28_Bjosephc_262e347f,2023-04-26 02:57:00,68710868,65,DEFAULT,3603.0,0.0,Thermo Fisher Scientific,Q Exactive HF,DONE,mzspec:MSV000091804:BJ2022111603
484284,MSV000091804/updates/2023-04-28_Bjosephc_262e3...,MSV000091804,peak,1,2023-04-28_Bjosephc_262e347f,2023-04-26 02:57:00,71670891,68,DEFAULT,3795.0,0.0,Thermo Fisher Scientific,Q Exactive HF,DONE,mzspec:MSV000091804:BJ2022111604


In [26]:
file_stats = file_stats.sort_values(by=["spectra_ms2"], ascending=[False]).drop_duplicates(["file_usi"])

In [27]:
file_stats.to_parquet(r"..\local_files\all_gnps_files\all_gnps_files.parquet", index=False)

In [28]:
file_stats = pd.read_parquet("../local_files/all_gnps_files/all_gnps_files.parquet", dtype=dtype)
file_stats["has_stats"] = True
file_stats.shape

(679308, 16)

In [29]:
from masst_utils import SPECIAL_MASSTS

rows = []
num_files = len(file_stats)
rows.append(
    {
        "spectra_ms1_sum": file_stats["spectra_ms1"].sum(),
        "spectra_ms2_sum": file_stats["spectra_ms2"].sum(),
        "spectra_ms1_mean": file_stats["spectra_ms1"].sum() / num_files,
        "spectra_ms2_mean": file_stats["spectra_ms2"].sum() / num_files,
        "size_mb": file_stats["size_mb"].sum(),
        "num_files": len(file_stats),
        "num_files_processed": len(file_stats[file_stats["file_processed"] == "DONE"]),
        "description": "ALL_GNPS/MASSIVE_mzML_mzXML"
    }
)

dfs = []
for masst in SPECIAL_MASSTS:
    file = masst.metadata_file
    df = pd.read_csv(file)
    df["masst"] = masst.prefix

    df = df.merge(file_stats, on="file_usi", how="left")
    num_files = len(df)
    has_stats = len(df[df["has_stats"] == True])
    rows.append(
        {
            "spectra_ms1_sum": df["spectra_ms1"].sum(),
            "spectra_ms2_sum": df["spectra_ms2"].sum(),
            "spectra_ms1_mean": df["spectra_ms1"].sum() / has_stats,
            "spectra_ms2_mean": df["spectra_ms2"].sum() / has_stats,
            "size_mb": df["size_mb"].sum(),
            "num_files": num_files,
            "num_files_processed": len(df[df["file_processed"] == "DONE"]),
            "num_files_with_stats": has_stats,
            "files_with_stats_percent": has_stats / num_files,
            "description": masst.prefix
        }
    )
    dfs.append(df)

sum_df = pd.DataFrame(rows)
sum_df.to_csv("../local_files/all_gnps_files/file_summary.csv", index=False)
sum_df

,spectra_ms1_sum,spectra_ms2_sum,spectra_ms1_mean,spectra_ms2_mean,size_mb,num_files,num_files_processed,description,num_files_with_stats,files_with_stats_percent
0,2.259182e+09,6.782255e+09,3325.711448,9984.064711,122272081.0,679308,376061,ALL_GNPS/MASSIVE_mzML_mzXML,NaN,NaN
1,2.062382e+06,8.665498e+06,585.072908,2458.297305,40819.0,3579,0,food,3525.0,0.984912
2,6.090121e+07,9.059531e+07,1002.356990,1491.084400,2797302.0,60781,10223,microbe,60758.0,0.999622
3,2.995789e+07,5.342205e+07,1718.853061,3065.124161,1038980.0,20209,10448,plant,17429.0,0.862438
4,3.715273e+07,8.715331e+07,1174.010428,2754.007142,1524156.0,32065,1116,gloabl,31646.0,0.986933


In [30]:
all_masst = pd.concat(dfs)
all_masst.to_csv("../local_files/all_gnps_files/file_summary_combined.csv", index=False)


In [31]:
all_masst[all_masst["has_stats"]==True]

,MassIVE,Filename,node_id,file_usi,masst,filepath,dataset,collection,is_update,update_name,...,DOIDCommonName,HealthStatus,LifeStage,NCBITaxonomy,SampleType,SampleTypeSub1,UBERONBodyPartName,DataSource,ID,ID_to_keep
2,MSV000084900,15NAVY01_v1_brk_1_GA4_01_39548.mzXML,peanut,mzspec:MSV000084900:15NAVY01_v1_brk_1_GA4_01_3...,food,MSV000084900/peak/Global_Foodomics_composite_d...,MSV000084900,peak,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MSV000084900,15NAVY01_v1_brk_2_GB4_01_39546.mzXML,grain/grass,mzspec:MSV000084900:15NAVY01_v1_brk_2_GB4_01_3...,food,MSV000084900/ccms_peak/Global_Foodomics_compos...,MSV000084900,ccms_peak,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MSV000084900,15NAVY01_v1_brk_3_GC4_01_39547.mzXML,grain/grass,mzspec:MSV000084900:15NAVY01_v1_brk_3_GC4_01_3...,food,MSV000084900/ccms_peak/Global_Foodomics_compos...,MSV000084900,ccms_peak,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MSV000084900,15NAVY01_v1_lun_1_GA5_01_39630.mzXML,complex,mzspec:MSV000084900:15NAVY01_v1_lun_1_GA5_01_3...,food,MSV000084900/peak/Global_Foodomics_composite_d...,MSV000084900,peak,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MSV000084900,15NAVY01_v1_lun_2_GB5_01_39627.mzXML,fruit,mzspec:MSV000084900:15NAVY01_v1_lun_2_GB5_01_3...,food,MSV000084900/ccms_peak/Global_Foodomics_compos...,MSV000084900,ccms_peak,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32060,NaN,NaN,NaN,mzspec:MSV000086989:NIST_POS_Samp_18-03,gloabl,MSV000086989/peak/mzXML_Files/NIST_POS_Samp_18...,MSV000086989,peak,0.0,NaN,...,No Information Collected,No Information Collected,No Information Collected,9606|Homo sapiens,animal,biofluid,feces,GNPS,feces_No Information Collected_9606|Homo sapie...,feces_No Information Collected_9606|Homo sapiens
32061,NaN,NaN,NaN,mzspec:MSV000086989:QC_Mix_01,gloabl,MSV000086989/peak/mzXML_Files/QC_Mix_01.mzXML,MSV000086989,peak,0.0,NaN,...,blank_QC,blank_QC,blank_QC,blank_QC,blank_QC,blank_QC,blank_QC,GNPS,blank_QC_blank_QC_blank_QC_blank_QC,blank_QC_blank_QC_blank_QC
32062,NaN,NaN,NaN,mzspec:MSV000086989:QC_Mix_02,gloabl,MSV000086989/ccms_peak/RAW_Files/QC_Mix_02.mzML,MSV000086989,ccms_peak,0.0,NaN,...,blank_QC,blank_QC,blank_QC,blank_QC,blank_QC,blank_QC,blank_QC,GNPS,blank_QC_blank_QC_blank_QC_blank_QC,blank_QC_blank_QC_blank_QC
32063,NaN,NaN,NaN,mzspec:MSV000086989:QC_Mix_03,gloabl,MSV000086989/peak/mzXML_Files/QC_Mix_03.mzXML,MSV000086989,peak,0.0,NaN,...,blank_QC,blank_QC,blank_QC,blank_QC,blank_QC,blank_QC,blank_QC,GNPS,blank_QC_blank_QC_blank_QC_blank_QC,blank_QC_blank_QC_blank_QC


In [33]:
file_stats.groupby("file_processed").count().reset_index()

,file_processed,filepath,dataset,collection,is_update,update_name,create_time,size,size_mb,sample_type,spectra_ms1,spectra_ms2,instrument_vendor,instrument_model,file_usi,has_stats
0,DONE,376061,376061,376061,376061,8490,376061,376061,376061,376061,376061,376061,376061,376061,376061,376061
1,FAILED,66903,66903,66903,66903,642,66903,66903,66903,66903,66903,66903,42,42,66903,66903
2,No,236344,236344,236344,236344,8332,236344,236344,236344,236344,235791,235796,233068,233068,236344,236344


In [44]:
plant_masst_missing = all_masst[(all_masst["has_stats"]!=True) & (all_masst["masst"]=="plant")]
plant_masst_missing.to_csv("../local_files/all_gnps_files/file_summary_plant_missing_stats.csv", index=False)
plant_masst_missing

,MassIVE,Filename,node_id,file_usi,masst,filepath,dataset,collection,is_update,update_name,...,DOIDCommonName,HealthStatus,LifeStage,NCBITaxonomy,SampleType,SampleTypeSub1,UBERONBodyPartName,DataSource,ID,ID_to_keep
187,MSV000079447,MSV000079447/-Green_RB4_01_11689.mzXML,NaN,mzspec:MSV000079447:-Green_RB4_01_11689,plant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,MSV000079447,MSV000079447/-Green_RC2_01_11685.mzXML,NaN,mzspec:MSV000079447:-Green_RC2_01_11685,plant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,MSV000079447,MSV000079447/-Green_RC3_01_11745.mzXML,NaN,mzspec:MSV000079447:-Green_RC3_01_11745,plant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,MSV000079447,MSV000079447/-Green_RC4_01_11696.mzXML,NaN,mzspec:MSV000079447:-Green_RC4_01_11696,plant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
191,MSV000079447,MSV000079447/-Green_RC5_01_11715.mzXML,NaN,mzspec:MSV000079447:-Green_RC5_01_11715,plant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20204,MSV000092992,MSV000092992/Tradescantia-pallida-Flower-ethyl...,NaN,mzspec:MSV000092992:Tradescantia-pallida-Flowe...,plant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20205,MSV000092992,MSV000092992/Vanilla-planifolia--ethylacetate-...,NaN,mzspec:MSV000092992:Vanilla-planifolia--ethyla...,plant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20206,MSV000092992,MSV000092992/Welwitschia-mirabilis--ethylaceta...,NaN,mzspec:MSV000092992:Welwitschia-mirabilis--eth...,plant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20207,MSV000092992,MSV000092992/Zamia-pumila--ethylacetate-2020.mzML,NaN,mzspec:MSV000092992:Zamia-pumila--ethylacetate...,plant,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
all_masst.groupby("masst").count().reset_index()

,masst,MassIVE,Filename,node_id,file_usi,filepath,dataset,collection,is_update,update_name,...,DOIDCommonName,HealthStatus,LifeStage,NCBITaxonomy,SampleType,SampleTypeSub1,UBERONBodyPartName,DataSource,ID,ID_to_keep
0,food,3579,3579,3579,3579,3525,3525,3525,3525,0,...,0,0,0,0,0,0,0,0,0,0
1,gloabl,0,0,0,32065,31646,31646,31646,31646,98,...,32065,32065,32065,32065,32065,32065,32065,32065,32065,32065
2,microbe,60781,60781,0,60781,60758,60758,60758,60758,978,...,0,0,0,0,0,0,0,0,0,0
3,plant,20209,20209,0,20209,17429,17429,17429,17429,2867,...,0,0,0,0,0,0,0,0,0,0
